In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#데이터 다운로드
from google.colab import drive
import os

path = '/content/drive/MyDrive/open (1)'

In [3]:
import pandas as pd
data_train = pd.read_csv(path + '/stk_hld_train.csv')
data_test = pd.read_csv(path + '/stk_hld_test.csv')

In [4]:
data_train.head()

,act_id,iem_cd,byn_dt,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324


In [5]:
data_test.head() # test 데이터에는 train데이터에 없는 hist_d 컬럼이 존재함, 2021년 이전에 주식을 보유한 기간 
# 힌드는 Hold_d ( 보유기간 ) - hist_d("2020~12.31 까지의 최근 보유기간) <= 146")
# 2021년 데이터 기간은 1.1~7.31로 제한, 다양한 기준으로 hist_d column을 만들어 새로운 Feature

''' 추가적으로 새로운 피쳐로 사용할 수 있는 정보에, 고객의 이전 주식 보유이력이 있다 
past_d / stk_bnc_hist.csv 데이터를 살펴보면 stk_hld_test.csv 고객들의 이전 보유이력 (past_d)를 확인가능
stk_hld_train.csv 도 마찬가지로 past_d column 만들수있음 '''

' 추가적으로 새로운 피쳐로 사용할 수 있는 정보에, 고객의 이전 주식 보유이력이 있다 \npast_d / stk_bnc_hist.csv 데이터를 살펴보면 stk_hld_test.csv 고객들의 이전 보유이력 (past_d)를 확인가능\nstk_hld_train.csv 도 마찬가지로 past_d column 만들수있음 '

In [6]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [18]:
cus = pd.read_csv(path + "/cus_info.csv")
iem = pd.read_csv(path + "/iem_info_20210902.csv")
hist = pd.read_csv(path + "/stk_bnc_hist.csv")
train = pd.read_csv(path + "/stk_hld_train.csv")
test = pd.read_csv(path + "/stk_hld_test.csv")

submission = pd.read_csv(path + "/sample_submission.csv")

In [19]:
print(len(cus))
print(len(iem))
print(len(hist))
print(len(train))
print(len(test))

10000
3079
2573839
681472
70596


In [20]:
# 앞서 언급한 것처럼 베이스라인에서는 "hist_d" column을 임의로 생성하여 모델을 학습시키도록 하겠습니다.
# 베이스라인에서는 "hold_d"값, 즉 주식 보유기간의 0.6배에 해당하는 기간을 임의로 설정하여 "hist_d"를 생성하였습니다.
# 결국 모델은 "hist_d"만큼 주식을 보유 했을때의 "hold_d"를 예측하게 될 것입니다.

train["hist_d"] = train["hold_d"]*0.6
train.hist_d = np.trunc(train["hist_d"])

In [21]:
# train과 test에 고객정보(cus_info)와 주식정보(iem_info)를 추가하겠습니다.

train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

In [22]:
train_data.head()

,act_id,iem_cd,byn_dt,hold_d,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99


In [23]:
train_data.isnull().sum()

act_id                  0
iem_cd                  0
byn_dt                  0
hold_d                  0
hist_d                  0
sex_dit_cd              0
cus_age_stn_cd          0
ivs_icn_cd              0
cus_aet_stn_cd          0
mrz_pdt_tp_sgm_cd       0
lsg_sgm_cd              0
tco_cus_grd_cd          0
tot_ivs_te_sgm_cd       0
mrz_btp_dit_cd          0
iem_krl_nm              0
btp_cfc_cd              0
mkt_pr_tal_scl_tp_cd    0
stk_dit_cd              0
dtype: int64

In [24]:
train.head()
train.duplicated("act_id")
print(train['act_id'].value_counts())
cus['act_id'].value_counts()

fd3dfcf13c21834d2bd707bafea9e6c4d17fcc21a90ee6491e6249555344231b    8266
6012ca0aa8039c4c282e0e63bcd6b223bcef0692883b90250303623444207d6b    4925
16ddab629cc1a23c6981e110ced28f4b0fa7b372ca534143c0dac4caea519a23    2948
79eb0719c09e59c1dd4ea0b77e5ab7c1a47069907b651d462eb26b03a369654a    2938
c597944745f39a90957ad86d088610e71ca807a9d2c9e4d1ee2508bb759c3395    2243
                                                                    ... 
67e7ae410c929aaf4b1789bc8048e5ec9b4e1d77990bcbded22bcdbfdf607ff3       1
83f1927c5969e158edfe118e0d8428ae83807ddc659eb476199040218a48f100       1
f9724f37751f408b4bbbe768b03b9a85a0ac5cad727bfdb8552d10771b26316a       1
31cffdb69fe18ca06cd0044007f8793e18c6d5d28dfeeb0a4811d17f3fb8b796       1
7058394c2c458f393d5a8438635ec66b9dc777afa3363dc402ce40d7481adad8       1
Name: act_id, Length: 9337, dtype: int64


e0c42f16f42a718231ba3213883680730c7d01042cfda0bdc5aa600a5ed76b30    1
bd12ae1b2eee534021f2bb27a83b1b5fc783825577caaa16d805855ea2d5dcb6    1
4abc63a8823f2613e10860e3c0639b81dcb56ca6558b5d94ee4c12aeedebe058    1
f0b1fefef7c033c09d93316c832a1931ba0de6eb8cc2127ad6c20c15dd8be548    1
ca66bbf3723e469952276780c185865e69615c3d62608fe1b630184af6445c5d    1
                                                                   ..
873814ca025ff88268392e1a207eac5c18a9c35c910f0608082a105b3596eaf3    1
f403dab4b3077045e0c02391dfdb30bf45b9fa803711045892477bc613070f3b    1
3c869a7413b8e622611c27f2b14d288f1367daee140249576c0786ab4d149c9d    1
081826f947b9fb3287fd9f85ab748d045f3f5ed715213b301963b724de8a92a5    1
db9f34814afba9309b3e703129a87c135b2ef8d30618cf9c85139de27e23d4f4    1
Name: act_id, Length: 10000, dtype: int64

In [25]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

In [27]:
print(train_data.head())
len(train_data)

                                              act_id   iem_cd    byn_dt  \
0  0ad104dbed99be0cd858aa772765ddedade554601a981b...  A006360  20180726   
1  0ad104dbed99be0cd858aa772765ddedade554601a981b...  A005930  20180131   
2  0ad104dbed99be0cd858aa772765ddedade554601a981b...  A005070  20180517   
3  0ad104dbed99be0cd858aa772765ddedade554601a981b...  A003520  20201112   
4  0ad104dbed99be0cd858aa772765ddedade554601a981b...  A002310  20180905   

   hist_d  sex_dit_cd  cus_age_stn_cd  ivs_icn_cd  cus_aet_stn_cd  \
0     6.0           1               9           3               2   
1    48.0           1               9           3               2   
2     3.0           1               9           3               2   
3    13.0           1               9           3               2   
4   194.0           1               9           3               2   

   mrz_pdt_tp_sgm_cd  lsg_sgm_cd  tco_cus_grd_cd  tot_ivs_te_sgm_cd  \
0                  2           9               5               

681472

In [28]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"] # 주가 
hist = hist.fillna(0) # 결측값 0으로 채우기 

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"]) # 사람과 종목이름이 같으면, 조인연산
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])] # 매수일자 = 기준일자 같은것만 train_data
train_data.reset_index(drop = True, inplace = True) # 행 인덱스 초기화

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [29]:
train_data.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,6.0,1,9,3,2,2,9,5,5,8,101,1,1,1,274.0,11782000.0,5000.0,43000.0
1,48.0,1,9,3,2,2,9,5,5,8,1361,9,1,1,2.0,4990000.0,5000.0,2495000.0
2,3.0,1,9,3,2,2,9,5,5,8,2530,12,2,99,786.0,14619600.0,1000.0,18600.0
3,13.0,1,9,3,2,2,9,5,5,8,1969,8,2,1,60.0,462000.0,500.0,7700.0
4,194.0,1,9,3,2,2,9,5,5,8,1696,10,3,99,183.0,8125200.0,5000.0,44400.0


In [30]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [31]:
train_data.head(1)

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,6.0,1,9,3,2,2,9,5,5,8,101,1,1,1,274.0,11782000.0,5000.0,43000.0


In [32]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 17.842	valid_0's rmse: 4.22398
[1000]	valid_0's l2: 16.624	valid_0's rmse: 4.07726
[1500]	valid_0's l2: 16.3445	valid_0's rmse: 4.04283
[2000]	valid_0's l2: 16.1574	valid_0's rmse: 4.01963
Did not meet early stopping. Best iteration is:
[1998]	valid_0's l2: 16.1555	valid_0's rmse: 4.01939
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 1.67409	valid_0's rmse: 1.29387
[1000]	valid_0's l2: 1.66703	valid_0's rmse: 1.29113
Early stopping, best iteration is:
[733]	valid_0's l2: 1.39706	valid_0's rmse: 1.18197
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 0.465223	valid_0's rmse: 0.682073
[1000]	valid_0's l2: 0.230714	valid_0's rmse: 0.480327
Early stopping, best iteration is:
[764]	valid_0's l2: 0.229322	valid_0's rmse: 0.478875
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 0.329605	valid_0's rmse: 0.5

In [33]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [34]:
submission["hold_d"] = np.round(predict)

In [35]:
submission.to_csv("dacon_baseline.csv", index = False)

In [50]:
train_data.head()


,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,6.0,1,9,3,2,2,9,5,5,8,101,1,1,1,274.0,11782000.0,5000.0,43000.0
1,48.0,1,9,3,2,2,9,5,5,8,1361,9,1,1,2.0,4990000.0,5000.0,2495000.0
2,3.0,1,9,3,2,2,9,5,5,8,2530,12,2,99,786.0,14619600.0,1000.0,18600.0
3,13.0,1,9,3,2,2,9,5,5,8,1969,8,2,1,60.0,462000.0,500.0,7700.0
4,194.0,1,9,3,2,2,9,5,5,8,1696,10,3,99,183.0,8125200.0,5000.0,44400.0


In [45]:
from sklearn.metrics import mean_squared_error
rmse_full_mod = mean_squared_error(model.predict(val_x), val_y, squared = False)

rmse_full_mod

0.50885013573449

In [48]:
# Initialize a list of results
results = []
# Iterate through each predictor
for predictor in val_x:
    
    # Create a copy of X_test
    X_test_copy = val_x.copy()
    
    # Scramble the values of the given predictor
    X_test_copy[predictor] = val_x[predictor].sample(frac=1).values
    
    # Calculate the new RMSE
    new_rmse = mean_squared_error(model.predict(X_test_copy), val_y,
                                  squared = False)
    
    # Append the increase in MSE to the list of results 
    results.append({'pred': predictor,
                    'score': new_rmse - rmse_full_mod })
# Convert to a pandas dataframe and rank the predictors by score
resultsdf = pd.DataFrame(results).sort_values(by = 'score',
                                              ascending = False)


In [49]:
resultsdf

,pred,score
0,hist_d,2.993059e+01
2,cus_age_stn_cd,2.859778e-02
17,stk_p,2.145931e-02
1,sex_dit_cd,1.187111e-02
9,mrz_btp_dit_cd,9.513528e-03
7,tco_cus_grd_cd,3.894071e-03
4,cus_aet_stn_cd,3.922477e-05
16,stk_par_pr,5.213474e-06
12,mkt_pr_tal_scl_tp_cd,5.069629e-06
13,stk_dit_cd,1.523411e-06


In [38]:
pip install eli5

     |████████████████████████████████| 106 kB 8.0 MB/s 
